# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

Student Name: Yuchao Wu
Studnet Number: 1000651984

In [1]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

     |████████████████████████████████| 471kB 2.9MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.4-cp36-cp36m-linux_x86_64.whl size=274549 sha256=9628bc9016a739b2f1c29bda0cfc0aa3ee64ed00b1169a7df03d067c65736010
  Stored in directory: /root/.cache/pip/wheels/58/30/73/8858a1b6e5e2674e2ea85c9904949c06addcf6fd34d59b5ea6
Successfully built pytrec-eval
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=dd706518632d692d7fc163c8739c06079e80322f330d32c061b96f2477d9b632
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government.zip'

In [3]:
!unzip government.zip

Archive:  government.zip
   creating: government/
  inflating: government/topics-with-full-descriptions.txt  
  inflating: government/gov.topics   
  inflating: government/gov.qrels    
   creating: government/documents/
   creating: government/documents/61/
  inflating: government/documents/61/G00-61-2800209  
  inflating: government/documents/61/G00-61-1192048  
  inflating: government/documents/61/G00-61-1118212  
  inflating: government/documents/61/G00-61-0749882  
  inflating: government/documents/61/G00-61-2230501  
  inflating: government/documents/61/G00-61-0680698  
  inflating: government/documents/61/G00-61-0551387  
  inflating: government/documents/61/G00-61-2575433  
  inflating: government/documents/61/G00-61-0469713  
  inflating: government/documents/61/G00-61-0280746  
  inflating: government/documents/61/G00-61-2574316  
  inflating: government/documents/61/G00-61-3933997  
  inflating: government/documents/61/G00-61-3290635  
  inflating: government/documents/61/G0

In [0]:
# imports
# Put all your imports here
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
from whoosh import scoring
from whoosh import qparser

In [0]:
DATA_DIR = "government"
#
# Put other path constants here
#
DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

Solution: MAP is considered as appropriate measure for the government web sites

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

Solution: The mean average precision measures the average precesion values at ranks of relevant documents. Bascially, it calculate average Precision across multiple queries or rankings. Its evluation considers the order of the documents. This is to say, a large MAP score indicates that there are many relevant docuemnts are ranked at the top of the retrieved results. This would in turn help users to find information they need easily and quickly.

## Question 2

### Q2 (a): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2

In [0]:
# Define a Create Index function
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()
    # create and return the index
    return index.create_in(indexDir, schema)

In [0]:
# Define a Schema for the index (The schema specifies the fields of documents in an index. Each doc can have multiple fileds: title, content, url,date)
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

In [0]:
# Create the index at the path INDEX_DIR based on the new schema
myIndex = createIndex(mySchema)

In [0]:
#Define a indexing file function
# open each docuemnt and index each docuemnt
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [0]:
# Build a list of files to index
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

In [0]:
# count files to index
print("number of files:", len(filesToIndex))

number of files: 4078


In [0]:
filesToIndex[0:5]

['government/documents/68/G00-68-3094820',
 'government/documents/68/G00-68-0361254',
 'government/documents/68/G00-68-2512403',
 'government/documents/68/G00-68-0000000',
 'government/documents/68/G00-68-1332243']

In [0]:
# Finishing the indexing
addFilesToIndex(myIndex, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
# Define a query parser for the field "file_content" in the index
myQueryParser = QueryParser("file_content", schema=myIndex.schema)

In [0]:
# Define a searcher
mySearcher = myIndex.searcher()

In [0]:
INDEX_Q2 = myIndex # Replace None with your index for Q2
QP_Q2 = myQueryParser # Replace None with your query parser for Q2
SEARCHER_Q2 = mySearcher # Replace None with your searcher for Q2

In [0]:
# Inspect the index
# Is it empty?
print("Index is empty?", INDEX_Q2.is_empty())

# How many files indexed?
print("Number of indexed files:", INDEX_Q2.doc_count())

Index is empty? False
Number of indexed files: 4078


In [0]:
# define a reader object on the index
myReader = INDEX_Q2.reader()

In [0]:
# Insepct the first 5 indexed documents
[(docnum, doc_dict) for (docnum, doc_dict) in myReader.iter_docs()][0:5]

[(0, {'file_path': 'government/documents/68/G00-68-3094820'}),
 (1, {'file_path': 'government/documents/68/G00-68-0361254'}),
 (2, {'file_path': 'government/documents/68/G00-68-2512403'}),
 (3, {'file_path': 'government/documents/68/G00-68-0000000'}),
 (4, {'file_path': 'government/documents/68/G00-68-1332243'})]

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

Solution for Q2(b): As shown below, the map all has value of 0.1971.

In [0]:
#Define a Trec evulation function
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [0]:
# run the above function to check the score for each measure
pyTrecEval(TOPIC_FILE, QRELS_FILE,QP_Q2,SEARCHER_Q2) 

Solution for Q2(b): As shown on the above resutl, the map all has value of 0.1971.

### Q2 (c): Provide answer to Q2(c) here [markdown cell]

The topics it did well are: 24 (1.0),18(1.0),


The topics it did poorly are: 28 (score = 0),16(0.0),9(0.00),7(0.00),6(0.0),2(0.0),1(0.0)

## Question 3

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

**What do you think would improve Whoosh’s performance on this test collection, and why?** - Solution: As shwon from example 1 to 6, the analyzers are reqruied to be updated to improve the whooh's performace. As it cannot understand the words in different forms (e.g., prefixes), failed to remove stop words and punctuation. 

Search System Evualtion: Take query 14, as an example. The document G00-79-4144643 is treat as a False Postive as it is not really relevant to the query but show up at the highest ranking. On the other hand, G00-89-0000000 (for example) is an FN because it is relevant to our search but not shown up in the ranking.

In [0]:
# Understand the topic file - quries
with open(TOPIC_FILE, "r") as f:
    print(f.read())

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education



In [0]:
# Example 1: Evaluate the originial anlayzer - General inspect
tokenizer = RegexTokenizer()
[token.text for token in tokenizer("I LOVE basketball, and football and all the sports.")]

['I', 'LOVE', 'basketball', 'and', 'football', 'and', 'all', 'the', 'sports']

As shown in example 1, the regular expression analyzer in the current search system has very limited filtering capbility (e.g., can only extract the words and igonore the white space without filtering out the stop words). A few adjustments to analyzer are requred to improve the overal performace of Whoosh performance. 

In [0]:
# Example 2: # Evaluate the originial anlayzer - Capitalization (topic 1)
print("# docs with 'mining'", myReader.doc_frequency("file_content", "mining"))
print("# docs with 'Mining'", myReader.doc_frequency("file_content", "Mining"))

# docs with 'mining' 45
# docs with 'Mining' 59


As shown in above example, the regular expression analyzer in the current research system are case sensitive (e.g, treat "Ming" and "ming" as two different words)

In [0]:
# Example 3: # Evaluate the originial anlayzer - Stop words (and, then, a)
print("# docs with 'and'", myReader.doc_frequency("file_content", "and"))
print("# docs with 'the'", myReader.doc_frequency("file_content", "the"))
print("# docs with 'a'", myReader.doc_frequency("file_content", "a"))

# docs with 'and' 3348
# docs with 'the' 3355
# docs with 'a' 2522


As shown in above example, the regular expression analyzer in the current research system cannot filter out the commmon stop words: and, the, a

In [0]:
# Example 5: # Evaluate the originial anlayzer - prefixes (Topic 4)
print("# docs with 'communications'", myReader.doc_frequency("file_content", "communications"))
print("# docs with 'commun'", myReader.doc_frequency("file_content", "commun"))

# docs with 'communications' 102
# docs with 'commun' 0


As shown in above example, the regular expression analyzer in the current research system cannot analyze the words in different form (e.g., 'communications' has a prefix form of 'commun'). In the real world, it is very likely for the users to enter the short version of the words instead of the full words.

In [0]:
# Example 6: # Evaluate the originial anlayzer - punctuation (Topic 19)
print("# docs with 'Cybercrime,'", myReader.doc_frequency("file_content", "Cybercrime,"))
print("# docs with 'Cybercrime'", myReader.doc_frequency("file_content", "Cybercrime"))

# docs with 'Cybercrime,' 0
# docs with 'Cybercrime' 12


As shown in above example, the regular expression analyzer in the current research system failed to remove the punctuations (e.g., 'Cybercrime,' vs'Cybercrime' ).

In [0]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

In [0]:
# FP - Irrelevant documents ranked highly
# FN - Relevant documents not ranked highly
printRelName(TOPIC_FILE, QRELS_FILE, myQueryParser, mySearcher, "14")

---------------------------Topic_id and Topic_phrase----------------------------------
14 Agricultural biotechnology
---------------------------Return documents----------------------------------
14 Q0 G00-79-4144643 0 16.017277 test
14 Q0 G00-09-1193469 1 15.942973 test
14 Q0 G00-45-0809730 2 15.640293 test
14 Q0 G00-89-0000000 3 15.377247 test
14 Q0 G00-88-1894712 4 8.188481 test
14 Q0 G00-34-0444524 5 3.239981 test
14 Q0 G00-84-0274223 6 1.544029 test
---------------------------Relevant documents----------------------------------
14 0 G00-89-0000000 1


The document G00-79-4144643 is treat as a false postive as it is not really relevant to the query but show up at the highest ranking. On the other hand, G00-89-0000000 is an FN because it is relevant to our search but not rank high in the ranking.

### Q3 (b): Write your code below

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

In [0]:
import nltk
from nltk.stem import *

In [0]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
# we'll compare two stemmers and a lemmatizer
lrStem = LancasterStemmer()
sbStem = SnowballStemmer("english")
wnLemm = WordNetLemmatizer()

In [0]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [0]:
#Define a modified analyzer
NewAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | CustomFilter(LancasterStemmer().stem)

In [0]:
# define a Schema with the new analyzer
mySchema2 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = NewAnalyzer))

In [0]:
# create the index based on the new schema
myIndex2 = createIndex(mySchema2)

In [0]:
addFilesToIndex(myIndex2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
myQueryParser2 = QueryParser("file_content", schema=myIndex2.schema)
mySearcher2 = myIndex2.searcher()

In [0]:
INDEX_Q3 = myIndex2 # Replace None with your index for Q3
QP_Q3 = myQueryParser2  # Replace None with your query parser for Q3
SEARCHER_Q3 = mySearcher2 # Replace None with your searcher for Q3

In [0]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser2, mySearcher2) 

**Solution:**  MAP_all = 0.3456 

In [0]:
# check the query from part a
printRelName(TOPIC_FILE, QRELS_FILE, myQueryParser2, mySearcher2, "14")

---------------------------Topic_id and Topic_phrase----------------------------------
14 Agricultural biotechnology
---------------------------Return documents----------------------------------
14 Q0 G00-89-0000000 0 18.326725 test
14 Q0 G00-79-4144643 1 16.239838 test
14 Q0 G00-97-1475424 2 15.661743 test
14 Q0 G00-01-3251318 3 15.661743 test
14 Q0 G00-51-1924264 4 15.457968 test
14 Q0 G00-72-0385489 5 15.132446 test
14 Q0 G00-86-2926152 6 14.131987 test
14 Q0 G00-45-0809730 7 14.003708 test
14 Q0 G00-09-1193469 8 13.867392 test
14 Q0 G00-36-2788975 9 13.594618 test
14 Q0 G00-97-2215955 10 13.402500 test
14 Q0 G00-09-1404763 11 13.278104 test
14 Q0 G00-10-2024294 12 13.127152 test
14 Q0 G00-91-1609512 13 13.099502 test
14 Q0 G00-06-2853218 14 12.474858 test
14 Q0 G00-35-2527252 15 12.412342 test
14 Q0 G00-06-0690672 16 12.165935 test
14 Q0 G00-86-0847220 17 11.425662 test
14 Q0 G00-88-1894712 18 11.319674 test
14 Q0 G00-70-3424520 19 10.954989 test
14 Q0 G00-07-2371962 20 9.207667 te

As shown above the relevant docuemnt G00-89-0000000 has been placed on the first rankning in the modefied system, whereas it ranked at 4th in the rankings for part a).

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

Additional filters were added to the current search system. For example, lowercaseFilter() was used to sense the words with lower cases. IntraWordFilter() was added to remove punctiona next to the words. Also, the StopFIlter() was added to remove the stop words. Finally, the NLTK stemmers were also added to the filter (e.g., landcaster stemmer). Overall, the MAP_all was improved from to 0.1971 to 0.3456 with the modifed analyzer.

In addition, the relevant docuemnt G00-89-0000000 (topic 14), was ranked the first place after using the modifed analyzer while it did rank high in the old analyzer.

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

YES

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

YES

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

Overall, it is evient that the modifed analyzer plays a significnat role in improving the system's performance in terms of retriving the relevant docuemnt. The MAP_all was imrpoved from 0.1971 to 0.3456 when applying the new analyzers. Some query topic, such as #14, shows an siginificant improvement and retrives the relevant docuement for the user whereas the origincial system failed to retrive the appropriate docuemnt. However, the udpated system still need some improvements for a better information retrial purpose.

## Question 4

In [0]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4

### Please answer the following questions here
(a) A clear list of all final modifications made.  
(b)  Why each modification was made – how did it help?  
(c)  The  final  MAP  performance  that  these  modifications  attained.

(a) Several Key modificaitons are: 1) Modifed Analyzer with lowercase Filter, intraworFilter,stopfilter,lancasterStemmer. 
2) Updated Query Parser: repalce the "or" algrithm instead of "AND". 
3) Upadted Scoring system: implemented BM25F with B =0.75 and several trails for K1 values ranging from 1.2 to 2.0.

(b) For the analzyer, the above-mentioned filters could do the functons, such as treating the upper and lower cases as same words, removing stop words, and reconiging the base form of the words. The purpose of this is to help the user retrieve the relevant information when indexed text contains words in different form than the one the user searches for.
The query with "OR" algrithm, instead of requiring all the terms present for a match, would only need any of the term present for a docuemnt to match. This would show more relevant and related results to the user.

For the scoring system, tuning the different paramers in BM25F scoring method (e.g., B and K1) would optimize the score for the research system. For example, when the text is a lot longer or divese, the K1 value of BM25F is suggested to be larger, which would increase the cap of term frequence saturation. Therefore, the user could find the docuemnts more quickly.



(c) The final MAP is 0.4026 when B = 0.55, K1 =2.55.

In [0]:
#Define a modified analyzer
NewAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | CustomFilter(LancasterStemmer().stem)

In [0]:
# define a Schema with the new analyzer
mySchemaQ4 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = NewAnalyzer))

In [0]:
# create the index based on the new schema
INDEXQ4 = createIndex(mySchemaQ4)

In [0]:
addFilesToIndex(INDEXQ4, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


From the Whoosh guide, the defualt QUeryParser would search for all the terms by defualt. For example, the defualt parser treates the words as if they were connected by AND, which means that all the terms must be present for a docuemnt to match. Thus, we need to change the parser to use OR such that any of the terms may be present for a docuemnt to match. Reference: https://whoosh.readthedocs.io/en/latest/parsing.html

In [0]:
# Define custimized Parser with OR instead of the default AND
myQueryParser3 = QueryParser("file_content", schema=INDEXQ4.schema,group=qparser.OrGroup)

For scoring system, the BM25F scoring will be used to improve the search performance. By defualt, the parameter of BM25F, b = 0.75 and k1 = 1.2, which would work well for the most of cases. For the purpose of the optimization, it would be worth tring some different values.



Reference:https://www.elastic.co/blog/practical-bm25-part-3-considerations-for-picking-b-and-k1-in-elasticsearch


In [0]:
# B = 0.75, K1 =1.2
mySearcher3_K1 = myIndex2.searcher(weighting = scoring.BM25F(B = 0.75, K1 =1.2))
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser3, mySearcher3_K1) 

map                      all     0.3797

In [0]:
# B = 0.75, K1 =1.5
mySearcher3_K2 = myIndex2.searcher(weighting = scoring.BM25F(B = 0.75, K1 =1.5))
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser3,mySearcher3_K2) 

map_all = 0.3810 @ k1 = 1.5

In [0]:
# B = 0.55, K1 =2.55
mySearcher3_K3 = myIndex2.searcher(weighting = scoring.BM25F(B = 0.55, K1 =2.55))
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser3,mySearcher3_K3) 

map_all = 0.4026 @ B = 0.55, K1 =2.55

In [0]:
# From above trails, we would store mySearcher3_K3 (k1 =2) as it has the highest score
INDEX_Q4 = INDEXQ4 # Replace None with your index for Q4
QP_Q4 = myQueryParser3 # Replace None with your query parser for Q4
SEARCHER_Q4 = mySearcher3_K3 # Replace None with your searcher for Q4

## Validation

In [0]:
# Run the following cells to make sure your code returns the correct value types

In [0]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [0]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [0]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation

In [0]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
